### In case that your kernel does not have the following libraries installed ...

In [7]:
# Packages for installation

# !python -m pip install allensdk
# !python -m pip install pynrrd
# !python -m pip install pandas
# !python -m pip install requests
# !python -m pip install matplotlib

In [1]:
%load_ext autoreload
%autoreload 2

from cfg import *
sys.path.append('libraries')

from mesoscale_extractor import NeuronPopulation
from utils import *
import convertAllenSpace as CAS

data_repository = 'atlas_files'

## Initialize the NeuronPopulation class with the requested source and target areas from which the statistics will be derived

In [2]:
neuropop_cls = NeuronPopulation(data_path = data_repository, res = 10)

## Create a meso-scale connectivity matrix comprised of anatomically distinct brain sub-areas, as defined by the Allen Reference Atlas

In [ ]:
layers = ['1','2/3','4','5','6a','6b']
source_areas = ['VPM','RT']  
target_areas = ['SSp-bfd'] 
layered_target_areas = [val1+val2 for val1 in target_areas for val2 in layers]
layered_target_areas = layered_target_areas + source_areas

mesoscale_stats_df = neuropop_cls.make_connectivity_matrix(source_areas, layered_target_areas, 
                                                            feature = 'length', extract = 'terminals') #mode = 'full', 

print((mesoscale_stats_df.loc[source_areas[0]]).mean(axis = 0, skipna = True))
print((mesoscale_stats_df.loc[source_areas[0]]).std(axis = 0, skipna = True))   


## Create a connectivity matrix with barrel-specific projections

In [ ]:
source_areas = ['VPM','RT']

# barrel_areas = ['Xa1','Xb2','Xc3']
# layered_barrel_areas = ['Xa1 L2/3','Xb2 L4','Xc3 L5']

barrel_annot, id2barrlabel = load_barrel_files(data_repository, islayer = True)
full_barrel_acros = sorted(list(id2barrlabel.values()))

barrel_stats_df = neuropop_cls.barrel_specific_matrix(source_areas, full_barrel_acros, feature = 'length', extract = 'terminals')

print((barrel_stats_df.loc[source_areas[0]]).mean(axis = 0, skipna = True))
print((barrel_stats_df.loc[source_areas[0]]).std(axis = 0, skipna = True))   


In [ ]:
from pdb import set_trace
alphabet = np.array(['a','b','c','d','e']) #'α','β', 'γ', 'δ',

projection_thr = 40
barrel_to_barreloid_dict = {}
for barrel_acro in full_barrel_acros:
    if 'L4' not in barrel_acro: continue
    barrel_acro_layerless = barrel_acro.split(' L4')[0]
    
    is_positive_proj = barrel_stats_df.loc['VPM'][barrel_acro] > projection_thr
    barreloid_neurons = barrel_stats_df.loc['VPM'][barrel_acro][is_positive_proj] 
    barreloid_neurons = list(barreloid_neurons.index)
    
    for barreloid_neuron in barreloid_neurons:
        l4_proj = barrel_stats_df.loc['VPM'][barrel_acro][barreloid_neuron]
        full_target_barrel_set = list(barrel_stats_df.loc['VPM', barreloid_neuron][barrel_stats_df.loc['VPM', barreloid_neuron] > 0].index)
        strongest_target = list(barrel_stats_df.loc['VPM', barreloid_neuron][barrel_stats_df.loc['VPM', barreloid_neuron] ==  barrel_stats_df.loc['VPM', barreloid_neuron].max()].index)[0]
        combined_strings = strongest_target[1] + ' ' + barrel_acro[1]
        crazy_condition = strongest_target[1] == barrel_acro[1] or ('a' in combined_strings and 'α' in combined_strings) or ('b' in combined_strings and 'β' in combined_strings) or ('c' in combined_strings and 'γ' in combined_strings) or ('d' in combined_strings and 'δ' in combined_strings)
        unique_trg_columns = np.unique([val[1:2] for val in full_target_barrel_set])      
        # if strongest_target[1] != barrel_acro[1]: continue
        if crazy_condition is False: continue
 
        unique_trg_columns[unique_trg_columns == 'γ'] = 'd'
        unique_trg_columns[unique_trg_columns == 'β'] = 'b'
        unique_trg_columns[unique_trg_columns == 'α'] = 'a'
        unique_trg_columns[unique_trg_columns == 'δ'] = 'd'

        if len(unique_trg_columns) > 1:
            column_distance = [np.where(alphabet == column)[0][0] for column in unique_trg_columns]
            max_col_dist = np.max([np.abs(val1 - val2) for val1 in column_distance for val2 in column_distance])
            if max_col_dist > 1:
                continue
        flag = True
        for val in ['L2/3','L5','L6a','L6b']:
            if barrel_acro.split(' L4')[0] + ' ' + val not in barrel_stats_df.columns: 
                continue
            another_layer_proj = barrel_stats_df.loc['VPM', barreloid_neuron][barrel_acro.split(' L4')[0] + ' ' + val]
            if another_layer_proj > l4_proj and np.isnan(another_layer_proj) == False:  
                flag = False
        if flag is True:
            if barrel_acro_layerless not in barrel_to_barreloid_dict:
                barrel_to_barreloid_dict[barrel_acro_layerless] = []
            barrel_to_barreloid_dict[barrel_acro_layerless].append(barreloid_neuron)         
    
    
# print(barrel_acro.split(' L4')[0], len(barrel_to_barreloid_dict[barrel_acro]))
sorted_keys = sorted(barrel_to_barreloid_dict, key=lambda x: len(barrel_to_barreloid_dict[x]))
barrel_to_barreloid_dict = {key: barrel_to_barreloid_dict[key] for key in sorted_keys[::-1]}

for barrel_acro, neuron_ids in barrel_to_barreloid_dict.items():
    print(barrel_acro, len(neuron_ids))


### TODO: Extract the barrel-layer picky-specific means and stds

In [ ]:
layer_acros = ['L1','L2/3','L4','L5','L6a','L6b']
picky_barrels = {}
for barrel_acro, neuron_ids in barrel_to_barreloid_dict.items():
    for layer_acro in layer_acros:
        barrel_layer_acro = barrel_acro + ' ' + layer_acro
        if barrel_layer_acro not in barrel_stats_df.columns: continue
        barrel_layer_proj_stats = barrel_stats_df[barrel_layer_acro]['VPM'][neuron_ids]
        barrel_layer_proj_stats[np.isnan(barrel_layer_proj_stats)] = 0
        means = barrel_layer_proj_stats.mean(skipna = True)
        stds = barrel_layer_proj_stats.std(skipna = True)
        enrichment = len(neuron_ids) #len(barrel_layer_proj_stats[barrel_layer_proj_stats > 0].index)
        if enrichment < 2 or np.isnan(means) or np.isnan(stds): continue
        means = np.round(means).astype(int)
        stds = np.round(stds).astype(int)
        print(barrel_layer_acro, means, stds, enrichment)
        picky_barrels[barrel_layer_acro] = {'mean' : means, 'std': stds, 'enrichment': enrichment}
        

In [ ]:
all_means, all_stds = {}, {}
for barrel_acro in barrel_stats_df.columns:
    means = barrel_stats_df[barrel_acro].mean(skipna = True)
    stds = barrel_stats_df[barrel_acro].std(skipna = True)
    if np.isnan(means) or np.isnan(stds): continue

    all_means[barrel_acro] = means
    all_stds[barrel_acro] = stds
    if barrel_acro in picky_stds.keys():
        print(barrel_acro, means, stds)

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure()
plt.boxplot([list(all_stds.values()), list(picky_stds.values())])
plt.show()

fig = plt.figure()
plt.boxplot([list(all_means.values()), list(picky_means.values())])
plt.show()

### TO DO: figure out the soma positions of these neurons, draw their convex hull and use it as a constraint for projections from the barrel cortex and RT

In [ ]:
from scipy.spatial import ConvexHull
out_orientation = out_orientation = ['um(10','PIR','corner']

sel_neuron_soma_cords = []
for barrel_acro, neuron_ids in barrel_to_barreloid_dict.items():
    for neuron_id in neuron_ids:
        db = 'mouselight' if 'AA' in neuron_id else 'braintell'
        neuronPath = 'https://neuroinformatics.nl/HBP/neuronsreunited-viewer/{}_json_gz/{}.json.gz'.format(db,neuron_id)
        file_content = requests.get(neuronPath).content
        target_neuron = json.loads(zlib.decompress(file_content, 16+zlib.MAX_WBITS))
        
        neuron_cls = NeuronMorphology(neuronDict = target_neuron)
        neuron_name = neuron_id.split('.')[0]
        if db == 'braintell':
            re_str = neuron_name.split('_')[1:3]
            neuron_name = re_str[0]+'_'+ re_str[1]
        neuron_cls.transform(out_orientation)
        print(neuron_cls.soma)
        sel_neuron_soma_cords.append(neuron_cls.soma)
        a = 1/0

sel_neuron_soma_cords = np.array(sel_neuron_soma_cords)
ConvexHull(sel_neuron_soma_cords)


### Note: for the reticular neurons, draw the convex hull of the proper barreloid neurons, then see which RT neurons target inside this convex hull, and select only their statistics. Accordingly, draw a convex hull for these RT neurons and select only feedback targets from the barrel cortex to the RT and VPM convex hulls respectively

## Concatenate the area-specific and barrel-specific matrices

In [ ]:
merge_stats_df = pd.concat([mesoscale_stats_df, barrel_stats_df], axis=1)

print((merge_stats_df.loc[source_areas[0]]).mean(axis = 0, skipna = True))
print((merge_stats_df.loc[source_areas[0]]).std(axis = 0, skipna = True))   
